# Импорт библиотек и загрузка данных

### Импортируем необходимые библиотеки для работы с данными (pandas для работы с таблицами данных, scikit-learn для машинного обучения) и загружаем данные из файла dataset_salary_2024_encoded.csv. Этот шаг необходим для подготовки данных к дальнейшему анализу и обучению моделей машинного обучения.

In [23]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv('dataset_salary_2024_encoded.csv')

X = df.drop(['salary_in_usd', 'salary_currency'], axis=1)  

y = df['salary_in_usd']

# Определение категориальных и числовых признаков и создание препроцессоров

### Анализируем структуру данных и определяем, какие признаки являются категориальными и какие числовыми. Для числовых признаков создаём преобразователь StandardScaler для их масштабирования, а для категориальных признаков применяем OneHotEncoder для преобразования в бинарные вектора.

In [24]:
categorical_features = ['experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_location', 'company_size']

numeric_features = X.drop(categorical_features, axis=1).columns

numeric_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Инициализация моделей и KFold разбиения

### Определяем модели, которые будем использовать для решения задачи регрессии (Linear Regression, Ridge Regression, Lasso Regression, ElasticNet Regression, Gradient Boosting). Настраиваем KFold для разбиения данных на обучающие и тестовые наборы с целью оценки производительности моделей на различных подмножествах данных.

In [25]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(max_iter=20000),
    'ElasticNet Regression': ElasticNet(max_iter=15000),
    'Gradient Boosting': GradientBoostingRegressor()
}

metrics = {
    'max_error': max_error,
    'mean_absolute_error': mean_absolute_error,
    'mean_squared_error': mean_squared_error,
    'r2_score': r2_score
}

# Обучение и оценка моделей с использованием KFold

### Для каждой модели и каждого фолда KFold разбиения обучаем модель на тренировочных данных, которые проходят предварительную обработку с использованием преобразователя данных (ColumnTransformer). Для каждого фолда вычисляем метрики производительности (max_error, mean_absolute_error, mean_squared_error, r2_score), которые сохраняем для последующего анализа.

In [26]:
results = {}

for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    fold_metrics = {'max_error': [], 'mean_absolute_error': [], 'mean_squared_error': [], 'r2_score': []}
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)
        
        y_pred = pipeline.predict(X_test)
        
        for metric_name, metric_func in metrics.items():
            metric_value = metric_func(y_test, y_pred)
            fold_metrics[metric_name].append(metric_value)
    
    results[model_name] = fold_metrics

    print(f"Results for {model_name}:")
    for metric_name, values in fold_metrics.items():
        mean_value = sum(values) / len(values)
        print(f"  {metric_name}: Mean = {mean_value}, Values = {values}")

Training Linear Regression...
Results for Linear Regression:
  max_error: Mean = 1209497.9727208652, Values = [370771.0271228127, 602217.7790280973, 3037649.2346759816, 537673.2428991026, 1499178.5798783326]
  mean_absolute_error: Mean = 8386.642932877097, Values = [8125.2564406649535, 8502.610484741577, 8918.095210563282, 8193.014974004427, 8194.237554411244]
  mean_squared_error: Mean = 1340253195.3713512, Values = [586027991.3145713, 856308128.9728245, 3465073068.900129, 668893831.2027202, 1124962956.4665103]
  r2_score: Mean = 0.7176465631266883, Values = [0.8774426742712551, 0.8071377894366911, 0.28545202247456003, 0.8567877980276575, 0.7614125314232775]
Training Ridge Regression...
Results for Ridge Regression:
  max_error: Mean = 927644.6514733136, Values = [396874.5549915937, 423615.8954083992, 3006613.0123117734, 425994.8545157601, 385124.9401390409]
  mean_absolute_error: Mean = 8716.799633510303, Values = [8730.205410056122, 8638.67828474177, 8952.263783066324, 8782.84510247

# Вывод результатов по каждой модели

### Анализируем результаты работы каждой модели. Для каждой модели выводим значения метрик производительности для каждого фолда KFold, что позволяет оценить стабильность и точность модели на различных подмножествах данных.

In [29]:
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    for metric_name, values in metrics.items():
        mean_value = sum(values) / len(values)
        print(f"{metric_name}: {values}")
        print(f"  Mean: {mean_value}")


Linear Regression:
max_error: [370771.0271228127, 602217.7790280973, 3037649.2346759816, 537673.2428991026, 1499178.5798783326]
  Mean: 1209497.9727208652
mean_absolute_error: [8125.2564406649535, 8502.610484741577, 8918.095210563282, 8193.014974004427, 8194.237554411244]
  Mean: 8386.642932877097
mean_squared_error: [586027991.3145713, 856308128.9728245, 3465073068.900129, 668893831.2027202, 1124962956.4665103]
  Mean: 1340253195.3713512
r2_score: [0.8774426742712551, 0.8071377894366911, 0.28545202247456003, 0.8567877980276575, 0.7614125314232775]
  Mean: 0.7176465631266883

Ridge Regression:
max_error: [396874.5549915937, 423615.8954083992, 3006613.0123117734, 425994.8545157601, 385124.9401390409]
  Mean: 927644.6514733136
mean_absolute_error: [8730.205410056122, 8638.67828474177, 8952.263783066324, 8782.845102471498, 8480.005587215803]
  Mean: 8716.799633510303
mean_squared_error: [673482918.9820064, 659024924.4298257, 3375059153.2476354, 599308800.710117, 523481276.281191]
  Mean:

# Выбор лучшей модели на основе метрики MSE и вывод её метрик

### Определяем наилучшую модель на основе метрики среднеквадратичной ошибки (MSE). Выбранная модель выводится вместе с её метриками производительности (max_error, mean_absolute_error, mean_squared_error, r2_score), что помогает принять окончательное решение о наиболее эффективной модели для данной задачи регрессии.

In [34]:
best_model_name = min(results, key=lambda x: sum(results[x]['mean_squared_error']) / len(results[x]['mean_squared_error']))

print(f"\nBest model: {best_model_name}")
print("Metrics:")
for metric_name, values in results[best_model_name].items():
    mean_value = sum(values) / len(values)
    print(f"{metric_name}: {values}")
    print(f"  Mean: {mean_value}")


Best model: Gradient Boosting
Metrics:
max_error: [527456.8710321651, 670719.4750401977, 363020.6786755883, 620255.0819876966, 424561.690809901]
  Mean: 521202.7595091097
mean_absolute_error: [1819.5548506236503, 1940.9607149183616, 1817.7047416878602, 1868.008827053433, 1755.0852609133467]
  Mean: 1840.2628790393305
mean_squared_error: [124173232.5762302, 274218920.01877767, 71268371.6162752, 179257886.079941, 97595662.14073665]
  Mean: 149302814.48639217
r2_score: [0.9740313781300811, 0.9382389757801972, 0.9853034352270946, 0.9616203418407138, 0.9793014500251941]
  Mean: 0.9676991162006562
